In [1]:
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms.functional as tf

In [2]:
# Number of Historical Days used as Input!
num_days_history = 14

In [3]:
class CustomDataset(Dataset):
    def __init__(self, start_pt, num_pts):
        self.start_pt = start_pt
        self.num_pts = num_pts
        self.factor = 10

        base = "data/"
        self.input_folder = base + "input/"
        self.output_folder = base + "output/"
        self.input_files = os.listdir(self.input_folder)
        self.output_files = os.listdir(self.output_folder)

        assert (start_pt + num_pts * self.factor) < len(self.output_files)

    def __len__(self):
        return self.num_pts

    def __getitem__(self, idx):
        idx = idx * self.factor
        input = []
        for i in range(num_days_history):
            input.append(torch.load(self.input_folder + self.input_files[self.start_pt + idx + 67 - i]))
        input = torch.cat(input, 0)
        output = torch.load(self.output_folder + self.output_files[self.start_pt + idx])
        return input, output

In [4]:
dataloader = DataLoader(CustomDataset(0, 12), batch_size=4, shuffle=True)
train_features, train_labels = next(iter(dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([4, 84, 676, 407])
Labels batch shape: torch.Size([4, 1, 676, 407])


Model

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        convs = []
        # conv_chs = [6 * num_days_history, 2048, 512, 128, 32, 8] # Can't execute this on local GPU!
        conv_chs = [6 * num_days_history, 64, 32, 16, 8]
        for i in range(len(conv_chs) - 1):
            convs.append(nn.Conv2d(conv_chs[i], conv_chs[i + 1], kernel_size=(3,3), padding="same"))
        self.convs = nn.ModuleList(convs)
        self.act = nn.ReLU()
        # self.LogSoftmax = nn.LogSoftmax(dim=1)
        self.final_conv = nn.Conv2d(conv_chs[-1], 1, kernel_size=(3,3), padding="same")
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        for conv in self.convs:
            x = self.act(conv(x))
        # x = self.LogSoftmax(x)
        x = self.sigmoid(self.final_conv(x))
        return x

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [7]:
# Building mask to remove invalid areas
file_path = "data/FD/" + os.listdir("data/FD/")[0]
im = Image.open(file_path)
mask = tf.to_tensor(im)
mask[mask >= 0] = True
mask[mask < 0] = False
mask = mask.bool().to(device)
print(f"Number of valid points:, {torch.sum(mask):,}")

Number of valid points:, 152,661


In [8]:
BATCH_SIZE = 4
train_dataloader = DataLoader(CustomDataset(0, 400), batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(CustomDataset(400, 40), batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(CustomDataset(440, 80), batch_size=BATCH_SIZE, shuffle=False)

In [9]:
train_steps = len(train_dataloader.dataset) // BATCH_SIZE
batch_mask = mask.repeat(BATCH_SIZE, 1, 1, 1)

In [10]:
weight = 50
eps = 1e-8
mask_sum = torch.sum(mask)
def weightedBCELoss(output, target):
    output = torch.clamp(output, eps, 1. - eps)
    loss = (weight * (target * torch.log(output))) + ((1 - target) * torch.log(1 - output))
    loss = loss * mask
    return -torch.sum(loss) / mask_sum

model = Net().to(device)
# loss_fn = nn.NLLLoss(torch.Tensor([1, 50]).to(device))
loss_fn = weightedBCELoss
optimizer = optim.Adam(model.parameters(), lr=1e-4)

print(model)

Net(
  (convs): ModuleList(
    (0): Conv2d(84, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (3): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
  )
  (act): ReLU()
  (final_conv): Conv2d(8, 1, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (sigmoid): Sigmoid()
)


In [11]:
def get_metrics(dataloader):
    model.eval()

    correct_num = 0
    correct_denom = 0
    recall_num = 0
    recall_denom = 0
    precision_num = 0
    precision_denom = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        y_pred = model(inputs)
        pred = y_pred > 0.5

        pred = pred[batch_mask]
        labels = labels[batch_mask]
        
        correct_num += (pred == labels).float().sum().item()
        correct_denom += labels.numel()
        recall_num += (pred[labels > 0] == labels[labels > 0]).float().sum().item()
        recall_denom += labels[labels > 0].numel()
        precision_num += (pred[pred > 0] == labels[pred > 0]).float().sum().item()
        precision_denom += labels[pred > 0].numel()
    
    correct = 0
    if correct_denom > 0:
        correct = correct_num / correct_denom
    recall = 0
    if recall_denom > 0:
        recall = recall_num / recall_denom
    precision = 0
    if precision_denom > 0:
        precision = precision_num / precision_denom
    
    return correct, recall, precision

In [15]:
best_model_value = 0

n_epochs = 200
nan_found = False
for epoch in range(n_epochs):
    model.train()

    total_train_loss = 0
    train_correct = 0
    train_recall = 0
    train_precision = 0
    for inputs, labels in tqdm(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)

        if torch.isnan(loss):
            print(f"NaN detected in loss!")
            torch.save(model.state_dict(), "model_nan.pt")
            nan_found = True
            break
        
        optimizer.zero_grad()
        loss.backward()

        # Check for NaNs in gradients
        for name, param in model.named_parameters():
            if param.grad is not None:
                if torch.isnan(param.grad).any():
                    print(f"NaN detected in gradients of {name}")
                    nan_found = True
        if nan_found == True:
            break
    
        optimizer.step()

        # Check for NaNs in model weights
        for name, param in model.named_parameters():
            if torch.isnan(param).any():
                print(f"NaN detected in weights of {name}")
                nan_found = True
        if nan_found == True:
            break

        total_train_loss += loss

        # pred = torch.argmax(y_pred, dim=1)
        pred = y_pred > 0.5

        pred = pred[batch_mask]
        labels = labels[batch_mask]
        
        train_correct += ((pred == labels).float().sum().item() / labels.numel())
        train_recall += ((pred[labels > 0] == labels[labels > 0]).float().sum().item() / labels[labels > 0].numel())
        if labels[pred > 0].numel() > 0:
            train_precision += ((pred[pred > 0] == labels[pred > 0]).float().sum().item() / labels[pred > 0].numel())
    
    if nan_found == True:
        break

    avg_train_loss = total_train_loss / train_steps
    train_correct = train_correct / train_steps
    train_recall = train_recall / train_steps
    train_precision = train_precision / train_steps
    
    print("Epoch: %d, Loss: %.4f" % (epoch+1, avg_train_loss))
    print("[Training Metrics]   Accuracy: %.4f, Recall: %.4f, Precision: %.4f" % (100*train_correct, 100*train_recall, 100*train_precision))

    val_correct, val_recall, val_precision = get_metrics(val_dataloader)

    print("[Validation Metrics] Accuracy: %.4f, Recall: %.4f, Precision: %.4f" % (100*val_correct, 100*val_recall, 100*val_precision))

    model_value = 0
    if (val_recall + val_precision) > 1e-4:
        model_value = (val_recall * val_precision) / (val_recall + val_precision) # F1 Score
    print("Model Value:", model_value)
    if model_value > best_model_value:
        print("Saving Model!")
        torch.save(model.state_dict(), "model.pt")
        best_model_value = model_value

100%|██████████| 100/100 [10:19<00:00,  6.20s/it]


Epoch: 1, Loss: 3.7438
[Training Metrics]   Accuracy: 69.8187, Recall: 81.5737, Precision: 5.1075
[Validation Metrics] Accuracy: 69.4475, Recall: 81.6051, Precision: 6.7359
Model Value: 0.062222820837959825


100%|██████████| 100/100 [10:16<00:00,  6.16s/it]


Epoch: 2, Loss: 3.7713
[Training Metrics]   Accuracy: 70.4100, Recall: 81.5707, Precision: 5.1846
[Validation Metrics] Accuracy: 63.5688, Recall: 87.9285, Precision: 6.0846
Model Value: 0.0569082535112298


100%|██████████| 100/100 [10:08<00:00,  6.08s/it]


...

100%|██████████| 100/100 [09:59<00:00,  6.00s/it]


Epoch: 28, Loss: 3.5978
[Training Metrics]   Accuracy: 72.5783, Recall: 81.1969, Precision: 5.4333
[Validation Metrics] Accuracy: 72.7917, Recall: 80.9453, Precision: 7.4639
Model Value: 0.06833758554214364


100%|██████████| 100/100 [10:05<00:00,  6.05s/it]


Epoch: 29, Loss: 3.5817
[Training Metrics]   Accuracy: 73.1534, Recall: 80.7692, Precision: 5.6863
[Validation Metrics] Accuracy: 68.5035, Recall: 84.4012, Precision: 6.7382
Model Value: 0.06240060066209344


 35%|███▌      | 35/100 [03:25<06:27,  5.96s/it]

In [14]:
print("Best Model Value:", model_value)

Best Model Value: 0.06233690708736561


In [19]:
torch.cuda.empty_cache()

In [15]:
checkpoint = torch.load("model.pt")
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [16]:
test_correct, test_recall, test_precision = get_metrics(test_dataloader)

print("[Test Metrics] Accuracy: %.4f, Recall: %.4f, Precision: %.4f" % (100*test_correct, 100*test_recall, 100*test_precision))

[Test Metrics] Accuracy: 79.0923, Recall: 75.4540, Precision: 7.6479


In [17]:
val_correct, val_recall, val_precision = get_metrics(val_dataloader)

print("[Validation Metrics] Accuracy: %.4f, Recall: %.4f, Precision: %.4f" % (100*val_correct, 100*val_recall, 100*val_precision))

model_value = 0
if (val_recall + val_precision) > 1e-4:
    model_value = (val_recall * val_precision) / (val_recall + val_precision) # F1 Score
print("Model Value:", model_value)

[Validation Metrics] Accuracy: 74.9329, Recall: 75.6927, Precision: 7.6187
Model Value: 0.06922002309941981


In [14]:
torch.save(model.state_dict(), "model_least_loss.pt")